## 对比 ( 直接drop非数值列  、 one-hot处理非数值列 ） 

1.drop 和 one-hot

2.align training set & test set

3.missing data 处理

4.cross validation

### data input

In [1]:
import pandas as pd
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [2]:
training_X = train_data.drop(['SalePrice'],axis=1)
y = train_data.SalePrice
test_X = test_data

### drop 和 one-hot


In [3]:
# Drop those non_numeric columns
numeric_training_X = training_X.select_dtypes(exclude=["object"]).drop("Id",1)
numeric_test_X = test_X.select_dtypes(exclude=["object"]).drop("Id",1)

# One-hot
one_hot_training_X = pd.get_dummies(training_X).drop("Id",1)
one_hot_test_X = pd.get_dummies(test_X).drop("Id",1)

### align training set & test set

In [4]:
one_hot_training_X, one_hot_test_X = one_hot_training_X.align(one_hot_test_X, join='left', axis=1)

### missing data 处理

In [5]:
def missing_data_handler(df):
    from sklearn.preprocessing import Imputer
    my_imputer = Imputer()
    cols_with_missing = (col for col in df.columns if df[col].isnull().any())
    for col in cols_with_missing:
        df[col+'_was_missing'] = df[col].isnull()
    df = my_imputer.fit_transform(df)
    return df

In [6]:
# directly drop
numeric_training_X = missing_data_handler(numeric_training_X)  
numeric_test_X = missing_data_handler(numeric_test_X) 

In [7]:
# one_hot_predictors
one_hot_training_X = missing_data_handler(one_hot_training_X)  
one_hot_test_X = missing_data_handler(one_hot_test_X) 

### Cross Validation Score 对比

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

def get_mae(X, y):
    # multiple by -1 to make positive MAE score instead of neg value returned as sklearn convention
    return -1 * cross_val_score(RandomForestRegressor(50), 
                                X, y, 
                                scoring = 'neg_mean_absolute_error').mean()

In [9]:
get_mae(numeric_training_X,y)

18404.239444092356

In [10]:
get_mae(one_hot_training_X,y)

17828.08139844461